In [1]:
import numpy as np

## 3.4 3층 신경망 구현하기

#### 3.4.1 표기법 설명

가중치 오른쪽 아래의 인덱스 번호는 '다음 층 번호, 앞 층 번호' 순으로 적는다.

### 3.4.2 각 층의 신호 전달 구현하기

#### 1. 은닉층(1층) 에서의 행렬 계산 / 활성화 함수 적용

In [2]:
X = np.array([1.0, 0.5]) # 입력층(0층) ndarray
W1 = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]]) # 은닉층(1층) 가중치 ndarray
B1 = np.array([0.1, 0.2, 0.3]) # 은닉층(1층) 바이어스 값 ndarray

print(X.shape)
print(W1.shape)
print(B1.shape)

(2,)
(2, 3)
(3,)


In [3]:
A1 = np.dot(X, W1) + B1 # 은닉층(1층) 가중치 합(가중 신호와 편향의 총합)
print(A1)
print(A1.shape)

[0.3 0.7 1.1]
(3,)


In [4]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x)) # 활성화함수 sigmoid(x)

In [5]:
Z1 = sigmoid(A1) # 활성화 함수 sigmoid를 통해 A1을 변환한 값 Z1
print(Z1)
print(Z1.shape)

[0.57444252 0.66818777 0.75026011]
(3,)


#### 2. 은닉층(2층) 에서의 행렬 계산 / 출력층(3층)으로 전달

In [6]:
# 2층에서의 X는 1층에서 만든 Z1값을 사용
W2 = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
B2 = np.array([0.1, 0.2])

print(Z1.shape)
print(W2.shape)
print(B2.shape)

(3,)
(3, 2)
(2,)


In [7]:
A2 = np.dot(Z1, W2) + B2 # 은닉층(2층) 가중치 합
print(A2)
print(A2.shape)

[0.51615984 1.21402696]
(2,)


In [8]:
Z2 = sigmoid(A2)
print(Z2)
print(Z2.shape)

[0.62624937 0.7710107 ]
(2,)


#### 3. 출력층(3층)에서의 출력

In [9]:
def identity_function(x):
    return x # 항등함수 : 입력을 그대로 출력하는 함수

In [10]:
W3 = np.array([[0.1, 0.3], [0.2, 0.4]])
B3 = np.array([0.1, 0.2])

A3 = np.dot(Z2, W3) + B3
Y = identity_function(A3) # 혹은 Y = A3

In [11]:
print(Y)
print(Y.shape)

[0.31682708 0.69627909]
(2,)


In [12]:
print("입력값 :\n{}".format(X))
print("출력값 :\n{}".format(Y))

입력값 :
[1.  0.5]
출력값 :
[0.31682708 0.69627909]


### 3.4.3 구현 정리   

신경망 구현의 관례에 따라 **가중치만 W1과 같이 대문자**로 쓰고, 그 외 편향과 중간 결과 등은 모두 소문자로 쓴다.

In [13]:
def init_network():
    network = {}
    network['W1'] = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['W2'] = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
    network['b2'] = np.array([0.1, 0.2])
    network['W3'] = np.array([[0.1, 0.3], [0.2, 0.4]])
    network['b3'] = np.array([0.1, 0.2])
    
    return network

def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    
    a3 = np.dot(z2, W3) + b3
    y = identity_function(a3)
    
    return y

network = init_network()
x = np.array([1.0, 0.5])
y = forward(network, x)
print(y)

[0.31682708 0.69627909]


## 3.5 출력층 설계하기

**회귀에는 항등 함수**를 사용하고, **분류에는 소프트맥스 함수**를 사용한다.

### 3.5.1 항등 함수와 소프트맥스 함수 구현하기

항등 함수는 입력을 그대로 출력한다. (항등 : 입력과 출력이 항상 같다는 뜻)

In [14]:
def identity_function(x):
    return x

소프트맥스 함수의 출력은 모든 입력 신호로부터 영향을 받는다.

In [15]:
a = np.array([0.3, 2.9, 4.0])

exp_a = np.exp(a) # 지수 함수
print(exp_a)

sum_exp_a = np.sum(exp_a) # 지수 함수의 합
print(sum_exp_a)

y = exp_a / sum_exp_a
print(y)
print(np.sum(y))

[ 1.34985881 18.17414537 54.59815003]
74.1221542101633
[0.01821127 0.24519181 0.73659691]
1.0


In [16]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

#### 3.5.2 소프트맥스 함수 구현 시 주의점

지수함수 특성 상 오버플로 문제가 발생한다.   
이를 막기 위해 입력 신호 중 최댓값을 이용한다.

In [17]:
a = np.array([1010, 1000, 990])
np.exp(a) / np.sum(np.exp(a)) # 오버플로 문제

C:\Users\morning\anaconda3\envs\week2\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
C:\Users\morning\anaconda3\envs\week2\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


array([nan, nan, nan])

In [18]:
c = np.max(a)
a - c

array([  0, -10, -20])

In [19]:
np.exp(a - c) / np.sum(np.exp(a - c))

array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09])

In [20]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # 오버플로 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y